# Operacje na modelach

## Agenda
1. Django shell
2. Pierwsze użycie modeli 
3. Więcej o modelu User
4. Panel administracyjny i rejestrowanie modeli
5. Querysety

### Django shell
Ale zaraz! Miało być przecież o modelach a nie o jakimś "szelu" ... Pokażę Ci, że `django shell` jest to bardzo przydatne narzędzie podczas tworzenia projektu w Django. `django shell` to tak naprawdę zwykła konsola pythonowa, z pewnymi dodatkami jak np. to, że jesteśmy połączeni z bazą danych więc możemy dodawać nowe rzeczy do bazy oraz wyciągać z niej przydatne informacje. 

Żeby włączyć zwykłą konsolę pythonową gdy jesteśmy w terminalu wystarczy wpisać `python` (czasami `python3`). W przypadku `django shell`'a które daje nam Django musimy wpisać komendę `python manage.py shell` czyli znowu korzystamy z django-admin CLI. Po wpisaniu powinien pokazać się pythonowy interpreter.

Stworzyliśmy ostatnio model `Article` w naszym projekcie, zaimportujmy sobie ten model, zaimportujmy również model użytkownika.
```
>>> from my_app.models import Article
>>> from django.contrib.auth.models import User
```

Jeśli nie wyrzuciło żadnego wyjątku np. `ModuleNotFoundError` to znaczy że wszystko jest super i możesz przejść do następnej sekcji.

### Pierwsze użycie modeli
W programowaniu tworzymy klasy po to, żeby na ich podstawie tworzyć obiekty, tak samo po to tworzymy modele, żeby na ich podstawie tworzyć obiekty, które nie tylko będą dostępne w naszym programie, ale również informacje o tych obiektach będą trzymane w bazie danych.

Modele mają metody, których możemy używać do różnych operacji na nich.
Pierwszą akcją jaką zademonstruje to będzie tworzenie obiektu:
Mam nadzieję, że dalej masz otwarty `shell` oraz masz zaimportowany modele `Article` oraz `User` jeśli tak, to wpisz:
```python
>>> user = User.objects.create_user(
     username="user999",
     password="password",
     email="test@gmail.com"
   )
>>> from django.utils import timezone
>>> article1 = Article.objects.create(
     title="tytuł ...",
     text="Opis ...",
     published_date=timezone.now(),
     author=user
    )
>>> article2 = Article.objects.create(
    title="inny tytuł",
    text="Inny opis",
    published_date=timezone.now(),
    author=user
   )
```

I właśnie dodałeś swoje pierwsze rekordy do bazy. Żeby sprawdzić, czy na pewno się wszystko dodało wpiszmy: 
```shell
>>> Article.objects.all()
>>> article1.title
>>> article1.published_date
```

Powinno zwrócić coś takiego:
```shell
>>> <QuerySet [<Article: Article object (1)>, <Article: Article object (2)>]>
>>> 'tytuł ...'
>>> datetime.datetime(2019, 9, 20, 14, 8, 31, 175887, tzinfo=<UTC>)
```

jeśli chcemy pobrać konkretny obiekt z bazy (np. o id = 2) to wpiszemy:
```shell
>>> article = Article.objects.get(id=2)
```

A co jeżeli będziemy próbowali uzyskać coś, czego nie ma w bazie ?
```shell
>>> Article.objects.get(id=1000)

```
dostajemy informacje:
```
    self.model._meta.object_name
my_app.models.Article.DoesNotExist: Article matching query does not exist.
```

więcej o wyciąganiu obiektów z bazy oraz filtrowanie tych obiektów w rozdziale o querysetach


### Więcej o modelu User
Jak już pewnie zauważyłeś kilka razy już korzystaliśmy z wbudowanego modelu `User`. Bardzo miło ze strony Django że zaimplementował nam taki model. Ma on wbudowane metody takie jak: `create_user()` lub `create_superuser()` które robią takie rzeczy za nas jak haszowanie hasła (**Trzymanie hasła jako string w bazie to jest bardzo głupi pomysł - nie należy tak robić!**)

Pola do jakich mamy dostęp przy modelu `User` to:
* id
* username
* password
* email
* first_name
* last_name
* date_joined
* last_login
* is_active
* is_staff
* is_superuser
* groups

A co jeśli chcielibyśmy dodać coś od siebie, ponieważ nie wystarczają nam pola które ten model nam oferuje ?
Powiedzmy że chcemy dodać polę `country` oraz `age` które będą odpowiadały za narodowość oraz wiek.
Mamy 2 opcje

1. **Napisanie własnego modelu użytkownika**

Napiszemy własny model, który będzie dziedziczył po: modelu: `django.contrib.auth.AbstractUser`. Taki napisany przez nas model, będzie miał wszystkie pola wyżej wymienione, oraz dodatkowe utworzone przez nas. Dodatkowo możemy nadpisać domyślne pola, np. ustawić `fist_name` oraz `last_name` jako pola obowiązkowe (wcześniej były opcjonalne)

```python
from django.db import models
from django.contrib.auth.models import AbstractUser


class MyUserModel(AbstractUser):
    # nadpisujemy istniejące pola
    first_name = models.CharField(max_length=40, null=False)
    last_name = models.CharField(max_length=40, null=False)
    # dodajemy nowe pola
    country = models.CharField(max_length=80)
    age = models.IntegerField()
```

Dodatkowo musimy powiedzieć django, że ten model będzie naszym domyślnym modelem użytkownika. W pliku `settings.py` tworzymy zmienną:

```python
# settings.py
AUTH_USER_MODEL = 'app_name.MyUserModel'
```

**Uwaga**
Możesz tak zrobić wyłącznie przed wykonaniem jakiejkolwiek migracji. Jeżeli w trakcie działania projektu zdałeś sobie sprawę, że chciałbyś tak - skasuj bazę i stwórz ją ponownie oraz usuń pliki migracyjne które są w folderze `migrates` w każdej aplikacji.

2. **Relacja OneToOne z modelem `User`**

Tworzymy normalny model, który dziedziczy tak jak zawsze z klasy `django.db.models.Model` jednym z pól będzie relacja z modelem `User` pozostałe pola, to dodatkowe pola:

```python
from django.db import models
from django.contrib.auth.models import User


class MyUserModel(models.Model):
    # powiązanie z modelem User
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    # dodajemy nowe pola
    country = models.CharField(max_length=80)
    age = models.IntegerField()
```

teraz traktujemy `MyUserModel` jako nasz domyślny. Żeby stworzyć użytkownika piszemy:

```python
MyUserModel.objects.create_user(
    username="user",
    password="somepassword"
    country="Poland",
    age=20
)
```


### Panel administracyjny i rejestrowanie modeli
Jesli tworzysz projekt zgodnie z tutorialem przejdź do pliku `urls.py` znajdującego się w folderze `new_project`
znajduje się tam następujący kod:
```python
from django.contrib import admin
from django.urls import path

from my_app import views

urlpatterns = [
    path('admin/', admin.site.urls), # <--- zwróc uwagę na tą linię
    path("", views.home_view, name="home"),
]
```

Pod adresem `http://localhost:8000/admin` mamy dostęp do panelu administracyjnego (jest on dostępny tylko dla adminów czyli takich userów którzy mają wartość pola `is_staff=True` lub `is_superuser=True`). Żeby tam przejść musisz mieć najpierw stworzonego użytkownika, który będzie adminem.


#### Tworzenie superużytkownika
Na poziomie pliku `manage.py` wpisz:
```
python manage.py createsuperuser
```
I wpisz wybrany przez ciebie login i hasło, którymi później będziesz wchodził na panel administarcyjny Django.

Moglibyśmy już włączyć serwer i przejść do tego adresu, jednak najpierw zarejestrujmy modele które utworzyliśmy:

#### Rejrestracja modeli
Żeby modele utworzone przez nas były widoczne w tym panelu admnistracyjnym musimy je **zarejestrować**
W aplikacji w której napisałeś modele przejdź do pliku `admin.py` i wpisz następujący kod:
```python
from django.contrib import admin
from my_app.models import Article

admin.site.register(Article)                            
```

Zarejestrowaliśmy model `Article`, teraz będzie widoczny w panelu administracyjnym.


Nareszcie możemy uruchomić serwer i sprawdzić czy wszystko działa poprawnie ;)
Uruchom serwer i przejdź na stronę: http:localhost:8000/admin
Wpisz login i hasło które wprowadziłeś tworząc `superusera` przed chwilą.
Jesteś w panelu administracyjnym! 

Jak widzisz oprócz modelu `Article` masz od razu zarejestrowane modele `User` oraz `Group`

Pobądź trochę w tym miejscu, spróbuj dodać jakieś obiekty z tego poziomu. To miejsce będzie twoim najlepszym przyjacielem podczas tworzenia projektu. Możesz na bieżąco śledzić jakie dane istnieją w bazie oraz manipulować tymi danymi.


### Querysety
Wiemy już jak tworzyć oraz poglądać nasze obiekty modeli. Pomimo tego, że panel administracyjny django jest bardzo użyteczny, musimy się też nauczyć wyciągać informacje z naszej bazy używając kodu. Do wyciągania konkretynych informacji używamy querysetów, czyli odpowiednio skontruowanych zapytań w składni rozumianej przez Django, które są potem konwertowane na język SQL i baza zostaje odpytywana. Gdy piszemy `Model.objects.all()` to jest to tłumaczone na `SELECT * FROM Model` i zostaje nam zwrocony wynik.

W django do pisania querysetów używamy obiektu `objects` który jest częścią każdego modelu. Potem definiujemy metodę w zależności od tego co chcemy osiągnąć np. jeżeli chcemy wyciągnąć wszystko uzywamy metody `all()`, jeżeli chcemy filtrować używamy metody `filter()` jeżeli chcemy wyciągnąć tylko pierwszy obiekt z bazy to używamy metody `first()`. Querysety mogą być bardzo proste np. `Model.objects.all()` ale mogą być bardzo zawaansowane np. chcemy wyciągnąć artykuły o id większym niż 1000 ale mniejszym niż 2000, w których data opublikowana jest później niż 20 lipca 2019 roku oraz tytuł zawiera słowo "python". Takie coś również możemy zrealizować w Django i wyglądałoby mniej więcej tak:

```python
Article.objects.filter(id__gte=1000, id__lte=2000, published_date__gt=datetime(2019, 7, 20), title__icontains="python")
```
To zapytanie zwraca listę który zawiera wszystkie obiekty pasujące do zapytania

Przedstawie kilka przykładów na podstawie stworzonego przez nas modelu `Article`

**Wyciąganie konkretnego obiektu**
```python
article_with_id_1 = Article.objects.get(id=1)
first_article = Article.objects.first()
first_article_different_way = Article.objects.all()[0] # wolniejszy sposób
```

**Filtrowanie po wartości pola**
```python
user = User.objects.get(id=1)
user_articles = Article.objects.filter(author=user)
```

**Filtrowanie z użyciem słów: większy niż / mniejszy niż / większy lub równy / mniejszy lub równy**
```python
# większy lub równy - gte
article_with_1000_or_more_likes = Article.objects.filter(likes__gte=1000)
# większy niż - gt
article_with_more_than_500_likes = Article.objects.filter(likes__gt=500)
# mniejszy lub równy - lte
article_with_1000_or_less_likes = Article.objects.filter(likes__lte=1000)
# mniejszy niż - lt
article_with_less_than_500_likes = Article.objects.filter(likes__lt=500)
```


**Filtrowanie żeby sprawdzić czy dane pole zawiera słowo**
```python
# wielkośc liter ma znaczenie:
article = Article.objects.filter(description__contains="python")
# wielkośc liter nie ma znaczenia:
article = Article.objects.filter(description__icontains="PyThOn")
```


**Sortowanie po ...**
```python
# sortowanie rosnąco po id
Article.objects.all().order_by("id")
# sortowanie malejąco po id
Article.objects.all().order_by("-id")
```

### Zadanie domowe
Jest to kontynuacja projektu my_project który był tworzony w zadaniu domowym w prezentacji nr. 4

1. Przejdź do shella i stwórz kilka użytkowników (instancje modelu `Author`) oraz stwórz kilka artykułów powiązanych z tymi użytkownikami
2. Zarejestruj modele w pliku `admin.py`
3. Stwórz superusera używając Django-CLI
4. Przejdź do panelu administracyjnego i stwórz kilka nowych artykułów oraz autorów
5. Przejdź ponownie do django shella i spróbuj wyciągnąć obiekty obiektów `Article` oraz `Author` używając sposobów wymienionych w 5 rozdziale

### Materiały
1. [Django shell vs python shell](https://stackoverflow.com/questions/37003872/difference-between-interactive-python-console-and-djangos-manage-py-shell)
2. [Django models (oficjalna dokumentacja)](https://docs.djangoproject.com/en/2.2/topics/db/models/)
3. [Więcej o modelach w django](https://www.tutorialspoint.com/django/django_models.htm)
4. [Django querysets (oficjalna dokumentacja)](https://docs.djangoproject.com/en/2.2/ref/models/querysets/)
5. [How to extend Django user model](https://simpleisbetterthancomplex.com/tutorial/2016/07/22/how-to-extend-django-user-model)